In [ ]:
import json
from io import StringIO

def read_geo_obj(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)
    
def print_key_trees(dict_obj):
    def print_indent(obj, indent=0):                
        if isinstance(obj, dict):            
            for k, v in obj.items():
                print('\t'*indent + k)
                print_indent(v, indent+1)
        elif isinstance(obj, list):
            print('\t'*indent + '[')
            print_indent(obj[0], indent+1)
            print('\t'*indent + ']')
    print_indent(dict_obj, 0)

In [ ]:
geo_obj = read_geo_obj('./input.geojson')

In [ ]:
print_key_trees(geo_obj)

In [126]:
def zipcode_latlon(features):
    buf = StringIO()
    buf.write("")
    for f in features:
        buf.write(f"double[] region_{f['properties']['ZIPCODE']}() "+"{\n" + "return new double[] ")
        lonlatList = f['geometry']['coordinates'][0][0]
        buf.write('{ ')
        isHead = True
        for lon,lat in lonlatList:
            if not isHead:
                buf.write(', ')
            buf.write(f"{lat}, {lon}")
            isHead = False
        buf.write('};\n}\n\n')
    return buf.getvalue()

In [ ]:
print(zipcode_latlon(geo_obj['features']))

In [127]:
def gisRegionMap(features):
    buf = StringIO()
    buf.write('public LinkedHashMap<String, GISRegion> __addGISRegionInLA(ShapeGISMap map) {')    
    buf.write("LinkedHashMap<String, GISRegion> regionsMap = new LinkedHashMap<String, GISRegion>();\n")
    feat_sorted = sorted(features, key=lambda f: f['properties']['ZIPCODE'])
    for f in feat_sorted:
        zipcode = f['properties']['ZIPCODE']
        buf.write(f'regionsMap.put("{zipcode}", new GISRegion(map, region_{zipcode}()));\n')
    buf.write('return regionsMap;\n')
    buf.write('}\n\n')
    return buf.getvalue()

In [128]:
def gisRegionMapWithFilter(features):
    buf = StringIO()
    buf.write('public LinkedHashMap<String, GISRegion> __addGISRegionInLA(ShapeGISMap map, List<String> onlyInclude) {')
    buf.write("LinkedHashMap<String, GISRegion> regionsMap = new LinkedHashMap<String, GISRegion>();\n")
    feat_sorted = sorted(features, key=lambda f: f['properties']['ZIPCODE'])
    for f in feat_sorted:
        zipcode = f['properties']['ZIPCODE']
        buf.write(f'if (onlyInclude.contains("{zipcode}")) regionsMap.put("{zipcode}", new GISRegion(map, region_{zipcode}()));\n')
    buf.write('return regionsMap;\n')
    buf.write('}\n\n')
    return buf.getvalue()

In [ ]:
print(gisRegionMap(geo_obj['features']))

In [ ]:
print(gisRegionMapWithFilter(geo_obj['features']))

In [ ]:
with open('init_RegionsMap.java', 'w') as f:
    f.write(zipcode_latlon(geo_obj['features']))
    f.write(gisRegionMap(geo_obj['features']))

In [129]:
def regionsCentMap(features):
    buf = StringIO()
    buf.write('public LinkedHashMap<String, Point> __addGISRegionCentersInLA(ShapeGISMap map) {')
    buf.write("LinkedHashMap<String, Point> regionsCentMap = new LinkedHashMap<String, Point>();\n")
    feat_sorted = sorted(features, key=lambda f: f['properties']['ZIPCODE'])
    for f in feat_sorted:
        zipcode = f['properties']['ZIPCODE']

        sumLat, sumLon = (0, 0)
        lonlatList = f['geometry']['coordinates'][0][0]
        nPoints = len(lonlatList)
        for lon,lat in lonlatList:
            sumLat += lat
            sumLon += lon
            
        centLat = sumLat / nPoints
        centLon = sumLon / nPoints
        
        buf.write(f'regionsCentMap.put("{zipcode}", (new Point()).setLatLon({centLat}, {centLon}));\n')
    buf.write('return regionsCentMap;\n')
    buf.write('}\n\n')
    return buf.getvalue()

In [ ]:
print(regionsCentMap(geo_obj['features']))

In [130]:
def regionsCentMapWithFilter(features):
    buf = StringIO()
    buf.write('public LinkedHashMap<String, Point> __addGISRegionCentersInLA(ShapeGISMap map, List<String> onlyInclude) {')    
    buf.write("LinkedHashMap<String, Point> regionsCentMap = new LinkedHashMap<String, Point>();\n")
    feat_sorted = sorted(features, key=lambda f: f['properties']['ZIPCODE'])
    for f in feat_sorted:
        zipcode = f['properties']['ZIPCODE']

        sumLat, sumLon = (0, 0)
        lonlatList = f['geometry']['coordinates'][0][0]
        nPoints = len(lonlatList)
        for lon,lat in lonlatList:
            sumLat += lat
            sumLon += lon
            
        centLat = sumLat / nPoints
        centLon = sumLon / nPoints
        
        buf.write(f'if (onlyInclude.contains("{zipcode}")) regionsCentMap.put("{zipcode}", (new Point()).setLatLon({centLat}, {centLon}));\n')
    buf.write('return regionsCentMap;\n')
    buf.write('}\n\n')    
    return buf.getvalue()

In [ ]:
print(regionsCentMapWithFilter(geo_obj['features']))

In [ ]:
def getCentralOfAllRegions(features):
    sumLat, sumLon = (0, 0)
    nPoints = 0
    for f in features:
        zipcode = f['properties']['ZIPCODE']
        lonlatList = f['geometry']['coordinates'][0][0]
        nPoints += len(lonlatList)
        for lon,lat in lonlatList:
            sumLat += lat
            sumLon += lon
            
    centLat = sumLat / nPoints
    centLon = sumLon / nPoints
    return centLat, centLon

In [ ]:
getCentralOfAllRegions(geo_obj['features'])

In [131]:
def addClassWrapper(*args):
    buf = StringIO()
    buf.write("""
/***************************
This file is auto generated by a bot.
***************************/
import java.util.Map;
import java.util.stream.Collectors;

public class InitGISRegions {
""")
    for arg in args:
        buf.write(arg)
        buf.write('\n')
    buf.write('}\n')
    return buf.getvalue()

In [133]:
feat = geo_obj['features']

with open('InitGISRegions.java', 'w') as f:
    f.write(
        addClassWrapper(
            zipcode_latlon(feat),
            gisRegionMap(feat),
            gisRegionMapWithFilter(feat),
            regionsCentMap(feat),
            regionsCentMapWithFilter(feat)
        )
    )

257901